<a href="https://colab.research.google.com/github/sc1abhis/BURGERBUILDER/blob/main/Acceptance_Probability_Score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_validate,GridSearchCV
from sklearn.metrics import roc_curve,roc_auc_score,confusion_matrix,classification_report,precision_score,recall_score,accuracy_score
import matplotlib.pyplot as plt
import imblearn
from imblearn.over_sampling import SMOTE

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_excel(r'/content/drive/MyDrive/Colab Notebooks/APS Algo/Input File2.xlsx',sheet_name = [0,1,2,3])

In [ ]:
data_1819 = data[0]
data_1920 = data[1]
data_2021 = data[2]
data_amcat = data[3]

In [ ]:
# data_1819 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/APS Algo/201819 Data.csv')
# data_1920 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/APS Algo/201920 Data.csv')
# data_2021 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/APS Algo/202021 Data.csv')
# data_amcat =pd.read_csv('/content/drive/MyDrive/Colab Notebooks/APS Algo/AMCAT Mapping - Old to New.csv')

In [ ]:
col = ['18-19 eng','18-19 sjt','19-20 eng','19-20 sjt','20-21 eng','20-21 sjt']
data_amcat = data_amcat[col]

In [ ]:
def essay_scores(df):
    fit_essay = df['Fit Essay Word Count']
    fit_essay_score = []
    for i in fit_essay:
        if i<=100:
            fit_essay_score.append(1)
        elif i>100 and i<=250:
            fit_essay_score.append(2)
        elif i>250 and i<=400:
            fit_essay_score.append(3)
        elif i>400 and i<=800:
            fit_essay_score.append(4)
        else :
            fit_essay_score.append(5)
    grit_essay = df['Grit Essay Word Count']
    grit_essay_score = []
    for i in grit_essay:
        if i<=100:
            grit_essay_score.append(1)
        elif i>100 and i<=250:
            grit_essay_score.append(2)
        elif i>250 and i<=375:
            grit_essay_score.append(3)
        elif i>375 and i<=750:
            grit_essay_score.append(4)
        else :
            grit_essay_score.append(5)
    return fit_essay_score,grit_essay_score

In [ ]:
col = ['Acceptance Probability Score (2019)','Acceptance Probability Score','Higher Secondary School Percentage','Higher Secondary School GPA Received', 'AMCAT English Score','AMCAT Situational Judgement Score','Grit Essay Word Count', 'Fit Essay Word Count','Undergraduate Final Percentage','Undergraduate Selectivity', 'Undergraduate Stream','Contact Master Status/Final Outcome','Leading Self Academic: Overall Score']
data_1819 = data_1819[col]       
data_1819.loc[:,'AMCAT English Score'] = data_amcat.loc[:,'18-19 eng']        
data_1819.loc[:,'AMCAT Situational Judgement Score'] = data_amcat.loc[:,'18-19 sjt']          
fit_score,grit_score = essay_scores(data_1819)       
#data_1819['Fit Essay Score'] = fit_score
#data_1819['Grit Essay Score'] = grit_score
data_1920 = data_1920[col]       
data_1920.loc[:,'AMCAT English Score'] = data_amcat.loc[:,'19-20 eng']        
data_1920.loc[:,'AMCAT Situational Judgement Score'] = data_amcat.loc[:,'19-20 sjt']          
fit_score,grit_score = essay_scores(data_1920)       
#data_1920['Fit Essay Score'] = fit_score
#data_1920['Grit Essay Score'] = grit_score
data_2021 = data_2021[col]       
data_2021.loc[:,'AMCAT English Score'] = data_amcat.loc[:,'20-21 eng']        
data_2021.loc[:,'AMCAT Situational Judgement Score'] = data_amcat.loc[:,'20-21 sjt']          
fit_score,grit_score = essay_scores(data_2021)       
#data_2021['Fit Essay Score'] = fit_score
#data_2021['Grit Essay Score'] = grit_score
final = data_1819.append(data_1920,ignore_index=True)
#temp = data_1819.append(data_1920,ignore_index=True)
#final = temp.append(data_2021,ignore_index=True)
temp2 = final.copy()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [ ]:
f,g = essay_scores(temp2)
temp2['Fit Essay Score']=f
temp2['Grit Essay Score']=g

In [ ]:
c = temp2.copy()
#c = c[col]
exclude = ['Phone Review- Recommend Assessment Center','Phone Review- No Show','First Review- Recommend Phone Review','Assessment Center- No Show','Withdrew from process','Assessment Center- No Signup','Ineligible']
count = 0
for i in c['Contact Master Status/Final Outcome']:
    if i in exclude:
        c.drop(count,inplace=True)
    count+=1
#print(c['Contact Master Status/Final Outcome'].unique())
status = []
for i in c['Contact Master Status/Final Outcome']:
    if "Reject" in i:
        status.append(0)
    else :
        status.append(1)
c['Status'] = status
#c['Contact Master Status/Final Outcome'].unique()

In [ ]:
col = ['Higher Secondary School Percentage','AMCAT English Score',
       'AMCAT Situational Judgement Score','Leading Self Academic: Overall Score', 'Fit Essay Score',
       'Grit Essay Score', 'Status']
df=c[col].dropna()
df.dropna(inplace=True)
df.isna().sum()

Higher Secondary School Percentage      0
AMCAT English Score                     0
AMCAT Situational Judgement Score       0
Leading Self Academic: Overall Score    0
Fit Essay Score                         0
Grit Essay Score                        0
Status                                  0
dtype: int64

In [ ]:
ytemp = df['Status']
x,y = SMOTE().fit_resample(df,ytemp)
col = ['Higher Secondary School Percentage','AMCAT English Score',
       'AMCAT Situational Judgement Score','Leading Self Academic: Overall Score', 'Fit Essay Score',
       'Grit Essay Score', 'Status']
x = pd.DataFrame(x , columns = col)
x.drop('Status',axis=1,inplace=True)
set(y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


{0, 1}

In [ ]:
x.shape

(40056, 6)

In [ ]:
fs = x['Fit Essay Score'].values
fes = pd.Categorical(fs,categories=[1,2,3,4,5],ordered=True)
fess = pd.Series(fes,x.index)
x['Fit Essay Score'] = fess
gs = x['Grit Essay Score'].values
ges = pd.Categorical(gs,categories=[1,2,3,4,5],ordered=True)
gess = pd.Series(ges,x.index)
x['Grit Essay Score'] = gess
ls = x['Leading Self Academic: Overall Score'].values
les = pd.Categorical(ls,categories=[1,2,3,4,5],ordered=True)
less = pd.Series(les,x.index)
x['Leading Self Academic: Overall Score'] = less

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.25)
X_train.shape
##X_train.dropna()


(30042, 6)

In [ ]:
## np.any(np.isnan(X_train))
## X_train.dropna(inplace=True)
##X_train = X_train.reset_index()

##X_train.replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
X_train.describe()

,Higher Secondary School Percentage,AMCAT English Score,AMCAT Situational Judgement Score
count,30042.000000,30042.000000,30042.000000
mean,77.800178,60.658467,55.045587
std,13.106766,25.279971,23.108584
min,0.000000,0.000000,0.000000
25%,68.804945,51.000000,48.366250
50%,79.509831,67.500000,63.000000
75%,88.897067,78.000000,69.000000
max,100.000000,98.000000,100.000000


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)
ypred = rfc.predict(X_test)
prob_rfc = rfc.predict_proba(X_test)[:,1]

ValueError: ignored

In [ ]:
col = ['Higher Secondary School Percentage','AMCAT English Score',
       'AMCAT Situational Judgement Score','Leading Self Academic: Overall Score', 'Fit Essay Score',
       'Grit Essay Score']
f,g = essay_scores(data_2021)
data_2021['Fit Essay Score']=f
data_2021['Grit Essay Score']=g
col.append('Contact Master Status/Final Outcome')
col.append('Acceptance Probability Score')
d = data_2021[col]
​
copy = d.copy()
copy['Contact Master Status/Final Outcome'].unique()
​
#exclude = ['Phone Review- Recommend Assessment Center','Phone Review- No Show','First Review- Recommend Phone Review','Assessment Center- No Show','Withdrew from process','Assessment Center- No Signup','Ineligible']
exclude = ['Phone Review- Recommend Assessment Center','First Review- Recommend Phone Review','Withdrew from process','Ineligible']
count = 0
for i in copy['Contact Master Status/Final Outcome']:
    if i in exclude:
        copy.drop(count,inplace=True)
    count+=1
#t = copy.copy()
status = []
for i in copy['Contact Master Status/Final Outcome']:
    if ("Reject"  in i) or ("No Show"  in i) or ("Signup"  in i):
        status.append(0)
    else :
        status.append(1)
copy['Status'] = status
#copy['Contact Master Status/Final Outcome'].unique()

In [ ]:
test = copy.dropna()
y_test = test['Status']
​
ypr = rfc.predict(test.drop(['Contact Master Status/Final Outcome','Acceptance Probability Score','Status'],axis=1))
​
#confusion_matrix(y_test,ypr)
#tn,fp,fn,tp = confusion_matrix(y_test,ypr).ravel()
#fn/len(ypr),fp/len(ypr),(tp+tn)/(tp+tn+fp+fn),tp,tn,fp,fn
yprob = rfc.predict_proba(test.drop(['Contact Master Status/Final Outcome','Acceptance Probability Score','Status'],axis=1))[:,1]